In [1]:
import PyPDF2
import pandas as pd
import pdfrw
import fitz
import os
import gspread

In [2]:
# Authenticate using the service account key
gc = gspread.service_account(filename="G:\My Drive\Source Code\Claims Form\coral-gate-380914-289ef7fcdd78.json")

# Open the Google Sheets document
sh = gc.open("Final Claims Form Automation")

# Select the worksheet by title
worksheet = sh.worksheet("Final CSV")

# Get all values from the worksheet
data = worksheet.get_all_values()

In [3]:
def fill_pdf(input_pdf_path, output_pdf_path, data_dict):
    template_pdf = pdfrw.PdfReader(input_pdf_path)
    for page in template_pdf.pages:
        annotations = page['/Annots']
        if annotations is None:
            continue
        for annotation in annotations:
            if annotation['/Subtype'] == '/Widget' and annotation['/FT'] == '/Tx':  # If it is a text field
                field_name = annotation['/T'][1:-1]  # Field name
                if field_name in data_dict:
                    field_value = data_dict[field_name]
                    if pd.isna(field_value):
                        field_value = ''  # Replace NaN with a blank value
                    annotation.update(pdfrw.PdfDict(V='{}'.format(field_value)))
    pdfrw.PdfWriter().write(output_pdf_path, template_pdf)

In [4]:
def set_checkboxes(pdf_path, output_pdf_path, sex_flag, rel_flag):
    # Define checkbox coordinates based on the flag
    checkbox_coords = {
        "rel_to_ins": (314.49798583984375, 156.7030029296875, 324.6080017089844, 166.81298828125),
        "employment": (307.97100830078125, 252.7540283203125, 318.08099365234375, 262.86199951171875),
        "pt_auto_accident": (308.7309875488281, 276.677978515625, 318.84100341796875, 286.7879943847656),
        "other_accident": (308.2179870605469, 299.9129943847656, 318.3280029296875, 310.02099609375),
        "ssn": (150.1580047607422, 684.2789916992188, 160.28700256347656, 694.3796997070312),
        "assignment": (287.1029968261719, 684.4320068359375, 297.21099853515625, 694.5291137695312),
        "insurance_type": (336.0329895019531, 108.333984375, 346.13299560546875, 118.43499755859375),
        "lab": (423.385009765625, 444.3659973144531, 433.4649963378906, 454.3389892578125),
        "sex": (314.4339904785156, 131.906005859375, 324.552001953125, 141.9420166015625),
        #"276": (441.26800537109375, 671.6900024414062, 451.92999267578125, 681.5250244140625),
    }

    if sex_flag != 'M':
        # Update checkbox coordinates for the non-'M' case
        checkbox_coords["sex"] = (350.28399658203125, 131.906005859375, 360.4020080566406, 141.9420166015625)
        
    if rel_flag == 'Self':
        # Update checkbox coordinates for the non-'M' case
        checkbox_coords["rel_to_ins"] = (249.91099548339844, 156.5059814453125, 260.02099609375, 166.614990234375)
        

    # Open the PDF
    pdf = fitz.open(pdf_path)
    
    # Remove the second page (if the document has more than one page)
    if len(pdf) > 1:
        page_to_remove = 1  # Page numbers are 0-based; 1 is the second page
        pdf.delete_page(page_to_remove)

    # Apply changes
    for page in pdf:
        # Get widgets (form fields)
        for widget in page.widgets():
            if widget.field_type == fitz.PDF_WIDGET_TYPE_CHECKBOX:
                for field_name, coords in checkbox_coords.items():
                    if widget.field_name == field_name:
                        target_rect = fitz.Rect(coords)
                        if widget.rect.intersects(target_rect):
                            widget.field_value = "Yes"  # Check the target checkbox
                        else:
                            widget.field_value = "Off"  # Uncheck other checkboxes with the same name
                        widget.update()

    # Save the modified PDF as a new file
    pdf.save(output_pdf_path)
    pdf.close()

    # Delete the temporary PDF file
    os.remove(pdf_path)

In [5]:
def sanitize_filename(filename):
    import re
    # Remove leading/trailing whitespace
    filename = filename.strip()
    # Replace newline characters with a space
    filename = re.sub(r'[\n\r]+', ' ', filename)
    # Replace invalid characters with '-'
    filename = re.sub(r'[\\/*?:"<>|]', '-', filename)
    # Replace multiple consecutive hyphens with a single one
    filename = re.sub(r'-+', '-', filename)
    return filename


In [6]:

df = pd.DataFrame(data)

# Extract the keys from the first row of the CSV
keys = df.iloc[0]

# For loop starts here from row 3 onwards till the end of the dataset
for i in range(2, len(df)):  # You can replace 20 with len(df) to iterate through the entire DataFrame
    
    # Extract the values from the current row of the CSV
    values = df.iloc[i]

    # Create a dictionary using keys and values
    data_dict = dict(zip(keys, values))
    
    # Additional fields
    data_dict['99icd'] = '0'
    data_dict['diagnosis1'] = 'F84.0'
    data_dict['ins_signature'] = 'Signature on File'
    data_dict['pt_signature'] = 'Signature on File'
    data_dict['doc_phone'] = '466 1305'
    data_dict['doc_phone area'] = '929'
    
    # Set sex_flag based on patient sex
    if data_dict['pt_sex'] == 'Male':
        sex_flag = 'M'
    else:
        sex_flag = 'F'
    
    # Set rel_flag based on patient sex
    if data_dict['rel_flag'] == 'Yes':
        rel_flag = 'Self'
    else:
        rel_flag = 'Child'
    
    # Fill the PDF with the data from the CSV
    input_pdf_path = r"G:\My Drive\Source Code\Claims Form\form-cms1500.pdf"
    output_pdf_path = r"G:\My Drive\Filled Claim Forms\CMS1500_filled_temp.pdf"
    fill_pdf(input_pdf_path, output_pdf_path, data_dict)
    
    ##
    output_file_name = f"{data_dict['claim_start_mm']} {data_dict['claim_start_dd']} - {data_dict['claim_end_mm']} {data_dict['claim_end_dd']} {data_dict['claim_yyyy']} {data_dict['claim_type']} {data_dict['pt_acr']} {data_dict['ins_policy']} {data_dict['claim_no']}"
    output_file_name = sanitize_filename(output_file_name)
    ##
    
    # Print the output filename to inspect its value
    print(output_file_name)
    
    ## output_file_name = f"{data_dict['claim_start_mm']} {data_dict['claim_start_dd']} - {data_dict['claim_end_mm']} {data_dict['claim_end_dd']} {data_dict['claim_yyyy']} {data_dict['claim_type']} {data_dict['pt_acr']} {data_dict['ins_policy']} {data_dict['claim_no']}"
    input_pdf_path = r"G:\My Drive\Filled Claim Forms\CMS1500_filled_temp.pdf"
    output_pdf_path = fr"G:\My Drive\Filled Claim Forms\{output_file_name}.pdf"
    set_checkboxes(input_pdf_path, output_pdf_path, sex_flag, rel_flag)


09 28 - 09 28 2024 BCBA Claim LuVu BCBS Anthem A10304000
08 29 - 08 29 2024 Para Claim IsCh Aetna B10304000
09 02 - 09 04 2024 Para Claim IsCh Aetna B10304001
10 04 - 10 04 2024 BCBA Claim OdCa BCBS MN B10304002
10 06 - 10 06 2024 BCBA Claim JoAn Allied B10304003
09 20 - 09 24 2024 Para Claim JaNol BCBS IBX B10314000
09 20 - 09 26 2024 Para Claim JaSh Aetna B10314001
09 20 - 09 26 2024 Para Claim JoGaj Aetna B10314002
09 27 - 10 03 2024 Para Claim JoIh Cigna B10314003
09 20 - 09 23 2024 Para Claim JuWh BCBS Anthem B10314004
08 26 - 08 28 2024 BCBA Claim EaPr Cigna C09174076
08 23 - 08 29 2024 BCBA Claim JaWa BCBS Regence C09184036
08 26 - 08 26 2024 BCBA Claim AiSh BCBS Anthem C09184079
09 06 - 09 11 2024 BCBA Claim KoHa Ambetter C10044002
09 06 - 09 11 2024 BCBA Claim LyOr Cigna C10044052
09 06 - 09 12 2024 BCBA Claim IsTy BCBS Anthem C10054012
09 06 - 09 11 2024 BCBA Claim JoSch BCBS Anthem C10054028
09 03 - 09 03 2024 Para Claim AvLa BCBS TN C10184005
09 04 - 09 04 2024 Para Claim A

10 09 - 10 09 2024 BCBA Claim ThCh BCBS Anthem F10304023
10 08 - 10 08 2024 BCBA Claim ThSh BCBS TN F10304024
10 09 - 10 09 2024 BCBA Claim ThSh BCBS TN F10304025
10 04 - 10 07 2024 BCBA Claim TiWin Ambetter F10304027
10 04 - 10 10 2024 BCBA Claim ToJo Ambetter F10304028
10 04 - 10 10 2024 BCBA Claim TrJa Ambetter F10304029
10 09 - 10 10 2024 BCBA Claim TrJoh BCBS Anthem F10304030
10 04 - 10 09 2024 BCBA Claim TrTu Ambetter F10304031
10 07 - 10 09 2024 BCBA Claim TuSp BCBS IL F10304032
10 08 - 10 09 2024 BCBA Claim ViPa Aetna F10304035
10 09 - 10 09 2024 BCBA Claim ViSa BCBS IL F10304036
10 09 - 10 10 2024 BCBA Claim WarEze Aetna F10304037
10 09 - 10 09 2024 BCBA Claim WeWa Cigna F10304038
10 08 - 10 08 2024 BCBA Claim WiCos BCBS Anthem F10304039
10 08 - 10 09 2024 BCBA Claim WiSu Ambetter F10304041
10 08 - 10 08 2024 BCBA Claim XaHa Champ VA F10304042
10 04 - 10 09 2024 BCBA Claim YaLe Ambetter F10304044
10 08 - 10 08 2024 BCBA Claim YoBo BCBS Anthem F10304045
10 04 - 10 04 2024 BCBA 

KeyboardInterrupt: 